# Fetch concordances and analyze them

Lars G. Johnsen, National Library of Norway

lars.johnsen@nb.no

Solstrand june 2021

### Initial code

In [39]:
import json
import pandas as pd
import requests
import spacy
from dhlab.module_update import css

def search_nb(word = 'demokrati', window=20, limit = 300):
    parameters = {
        'query': word,
        'window':window,
        'limit':limit
    }
    r = requests.get("https://api.nb.no/ngram/db1/konk", params = parameters)
    return pd.DataFrame(json.loads(r.text), columns = 'doc-id concordance'.split())

In [24]:
import spacy

nlp = spacy.load("nb_core_news_sm")

In [27]:
def parse(string):
    cols = "token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop".split(', ')
    doc = nlp(string)
    rows = [(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop) for token in doc]
    return pd.DataFrame(rows, columns = cols), doc

In [40]:
pd.set_option('max_colwidth', None)
css()

# Concordances

A random collection of newspapers, periodicals and/or books are created on each query. Use SQLite fts5 syntax in formulating search https://www2.sqlite.org/fts5.html

In [76]:
df = search_nb(""" "boken han" """, limit = 200)
df.style

,doc-id,concordance
0,100200065,"... forhindret ham i å få noen mening ut av boken han lå og leste i , - så kan vi ligge litt..."
1,100122276,... I mangel av bedre råd åpnet han derfor boken han bar på og begynte å deklamere .
2,100122276,"Andreas så opp fra boken . Han hadde fått besøk - av pater Leo , på vei til eller fra et eller annet ."
3,100242714,"... det var som de meldte sig i logisk rekkefolge , og føiet sig villig inn i boken han holdt på med."
4,100081702,... Han forteller Irakli om boken han leser . Han
5,100081702,"Irakli åpnet boken . Han leste ett dikt , så lukket han boken og la hånden på permen . Han så opp i..."
6,100064151,"... Borges ble derfor dobbelt overrasket over å oppdage at boken han hadde begynt å lese , var skrevet på et språk..."
7,100162420,... som nok er opphavsmann for mange av vendingene i denne boken . Han vet mer om digitalt fjernsyn enn noen andre .
8,100540465,"... Hovmesteren går bort til podiet og studerer boken . Han får en telefon og legger på igjen etter få sekunder , deretter..."
9,100422335,"... Der på nattbordet hans sto fotografiet av henne seiv i sin smale bronseramme , der lå den siste boken han hadde..."


# Parse using Spacy

## Collect all the concordances 

These are all contained in the variable `df`

The function parse returns the parse as a data frame or as a spacy doc

In [61]:
# remove the phrase markers and parse the sentences

parses = [parse(s.replace("<b>", "").replace("</b>","")) for s in df['concordance']]

df_versions = [p[0] for p in parses]
spacy_docs = [p[1] for p in parses]

In [66]:
df_parses = pd.concat(df_versions)
df_parses

,token.text,token.lemma_,token.pos_,token.tag_,token.dep_,token.shape_,token.is_alpha,token.is_stop
0,...,...,PUNCT,PUNCT,ROOT,...,False,False
1,Og,og,CCONJ,CCONJ,cc,Xx,True,True
2,hvordan,hvordan,ADV,ADV,advmod,xxxx,True,True
3,går,gåre,VERB,VERB,ROOT,xxx,True,True
4,det,det,PRON,PRON,expl,xxx,True,True
...,...,...,...,...,...,...,...,...
18,bred,bred,ADJ,ADJ,amod,xxxx,True,False
19,fremstilling,fremstilling,NOUN,NOUN,obj,xxxx,True,False
20,av,av,ADP,ADP,case,xx,True,True
21,sydafrikansk,sydafrikansk,ADJ,ADJ,nmod,xxxx,True,False


## Check out frequency of some tags

In [69]:
df_parses.groupby("token.pos_").count()['token.text'].sort_values(ascending=False)

token.pos_
PUNCT    465
NOUN     398
PRON     331
VERB     326
ADP      288
ADJ      171
DET      131
AUX      129
PROPN    124
SCONJ     76
CCONJ     75
ADV       60
PART      41
NUM       25
X          9
INTJ       4
SYM        1
Name: token.text, dtype: int64

In [70]:
df_parses.groupby("token.dep_").count()['token.text'].sort_values(ascending=False)

token.dep_
punct           410
nsubj           306
ROOT            288
case            243
obl             168
obj             130
advmod          123
nmod            117
det             115
conj             90
mark             84
aux              76
amod             72
cc               70
acl:relcl        60
cop              52
compound:prt     34
flat:name        34
advcl            29
xcomp            28
dep              21
expl             20
ccomp            19
nummod           15
acl              13
iobj              8
parataxis         8
appos             7
csubj             3
compound          3
discourse         2
nsubj:pass        2
flat:foreign      1
aux:pass          1
orphan            1
acl:cleft         1
Name: token.text, dtype: int64

## Which words are in certain relations?

Searching using standard pandas expressions

In [75]:
df_parses[df_parses['token.dep_'] == 'nsubj'].groupby('token.text').count()['token.lemma_'].sort_values(ascending=False).head(20)

token.text
han         93
Han         57
jeg         10
som          8
boken        7
Det          5
Hun          4
De           4
du           3
Jeg          3
det          2
John         2
ham          2
man          2
mange        2
alle         2
den          2
Boken        2
vi           2
tiltalte     2
Name: token.lemma_, dtype: int64

# Se på et tre

In [51]:
df.concordance[6]

'... Han hadde begynt å lese i Nestors bok , den eneste <b>boken han</b> noensinne hadde sett Nestor lese i . Da Cesar... '

In [65]:
spacy.displacy.render(spacy_docs[6])